Reflector Grain Orientations
============================



## Introduction



The effect of subduction alignment of magnetisation can be inferred from the orientation of reflective and elongated magnetite grains.

With reflector grain geometries found in the RL scan processing notebooks, their long axis orientations and areas can be passed through filters to plot rose diagrams or isolate and display grain orientation populations. The grain contours are stored in expanded pixel coordinates where each pixel is 0.5 microns by 0.5 microns.

The underlying proceedure/algorithm here was used in `grain-orientation-checking.pdf` of the Supplementary Materials.



## Data Loading



The large volume of grains in each sample means grain data is stored per sample. As such this method can only be reasonably applied to one sample at a time. The first step is to load the contours.



In [1]:
import numpy as np
import os

# Declare sample for processing.
sample = "07A"
# Declare desired reflector grain "denoising" kernel size used to process the sample (required to construct the datapath).
kernel_px = 3
# Construct path to contours data folder for the requested kernel size.
contours_path = f"../reflector_processing/section-scans-refined-full/contours-modified-{kernel_px}"
# Load contours data for the requested sample, Note - contours will be enlarged by a factor of 2.
contours = np.load(os.path.join(contours_path,f"{sample}.png-larger.npy"),allow_pickle=True)

## Filtering



Angle, minimum elongation and maximum area filter specifications can be declared for the sample. The maximum area filter is treated as a within-sample variable, with grains passing the angle and minimum elongation filters still being recorded even if they fail the area filter (i.e. only angle and elongation filters are constant for the sample).

-   The area filter acts on the bounding box rather than grain area strictly (for quicker code running) since the bounding box tends to be a good approximation.



In [1]:
# Declare orientation filter.
angle_range = np.radians(np.array([35,55]))
# Declare minimum elongation filter.
min_elongation = 2
# Declare maximum area filters.
max_areas = [100,1000]

Each grain's contour can then be analysed individually, and orientations (angle clockwise from horizontal, with range -180 to 180) collated.



In [1]:
import cv2
from shapely import oriented_envelope
from tqdm import tqdm

# Initialize storage for grain elongation vector line definitions.
elongation_lines = {a:[] for a in max_areas}
# Initialize storage for non-orientation filtered grain orientations.
rotations = {a:[] for a in max_areas + ["all"]}
# Initialize storage for grain bounding rectangles.
rectangles = {a:[] for a in max_areas}
# Iterate through the contours and display progress.
for contour in tqdm(contours):
    # Fit a rotated rectangle to the contour.
    center,dimensions,rect_rotation = cv2.minAreaRect(contour)
    # Find rectangle vertex coordinates.
    p0,p1,p2,p3 = cv2.boxPoints((center,dimensions,rect_rotation))
    # Compute rectangle axes lengths.
    dist_0_1 = np.linalg.norm(p0-p1)
    dist_0_3 = np.linalg.norm(p0-p3)
    # Determine the long axis for the corresponding point p_long that creates the longest line with p0.
    if dist_0_1 > dist_0_3:
        p_long = p1
    else:
        p_long = p3
    # Compute the difference in coordinates represented by the longest line.
    dx,dy = (p0-p_long)
    # Fix coordinate convention for angle clockwise horizontal.
    dy = -dy
    # Compute rotation of the grain.
    # Non-directional orientation, so arctan is fine.
    rotation = np.arctan(dy/dx)
    # Compute area from bounding box dimensions.
    # Note halving of dimensions to get area in microns^2.
    area = ((dimensions[0]/2) * (dimensions[1]/2))
    # Construct filter for grains with rotations that are in the desired range (with angle range going clockwise from the first to second angle).
    # This conditional handles ranges spanning across 0 degrees.
    if angle_range[1] > angle_range[0]:
        angle_filter = (rotation >= angle_range[0]) and (rotation <= angle_range[1])
    else:
        angle_filter = (rotation >= angle_range[0]) or (rotation <= angle_range[1])
    # Check if the grain passes the minimum elongation filter.
    if (max(dimensions)/min(dimensions) > min_elongation):
        # Iterate through the maximum area filters.
        for max_area in max_areas:
            # Check if the grain passes the maximum area filter.
            if area < max_area:
                # Store computed grain rotation under the filter.
                rotations[max_area].append(rotation)
                # Check if the grain passes the orientation filter.
                if angle_filter:
                    # Store grain orientation vector specification under the filter.
                    # Note the halving of the center coordinates to ensure they are in the units of microns. The vector lengths in the x and y axes are reduced so that they aren't excessively long when plotted.
                    length_reduction_factor = 100
                    elongation_lines[max_area].append(np.array([*np.array(center)/2,dx/length_reduction_factor,dy/length_reduction_factor]).flatten())
                    # Store grain rectangle specification under the filter.
                    rectangles[max_area].append(np.array([p0,p1,p2,p3,p0])/2)
            else:
                # Store orientations that passed the elongation filter only.
                rotations["all"].append(rotation)

With filtered grain orientations and characteristic geometries (bounding box and orientation vector) found, they can now be plotted only the RL scan (1 px = 1 micron) for visual inspection. The maximum area of interest must be declared here (and can be changed between reruns of this code block).



In [1]:
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib as mpl

mpl.use("TkAgg")

# Declare maximum area of grains for this code block/plotting.
use_max_area = max_areas[0]

# Declare the path to the RL scan images that were also used to detect grains from.
imgs_path = "../DATASETS/RL_scans/"
# Load and plot RL scan.
img = Image.open(imgs_path+sample+".jpg")
plt.imshow(img)
# Plot orientation vectors.
plt.quiver(*np.array(elongation_lines[use_max_area]).T,headwidth=0.1,scale=1,color="lightblue",label=f"<{use_max_area:d} micron$^2$ grains")
# Plot grain rectangles.
bounding_rects = [Polygon(coords,closed=True,edgecolor="r",fill=False,linewidth=2,zorder=100) for coords in rectangles[use_max_area]]
p = PatchCollection(bounding_rects,match_original=True)
plt.gca().add_collection(p)
# Set axes aspect ratio to 1:1.
plt.gca().set_aspect("equal")

plt.show()

None

These plots can be used to characterize the spatial distribution of grains oriented in a certain direction.



## Rose Plots



A more quantitative way to check for common (modal) orientations is through the use of a rose plot. Bars on the rose plot can be overlain on each other in order of decreasing maximum area (filter) to permit characterization of narrower grain populations on the same plot.



In [1]:
# Ensure max area filter order is from small to large.
max_area_order = sorted(max_areas) + ["all"]

import matplotlib.pyplot as plt

def plot_half_rose(orientations,ax,n_bins,**plot_kwargs):
    ''' Produce a half rose diagram from list of orientation datapoints.

    orientations | :list:-like | List of orientation measurements (angles).
    ax | :matplotlib.projections.polar.PolarAxes: | Stereonet axis to plot the rose diagram on.
    n_bins | :int: | Number of bins to group the orientations into.
    plot_kwargs | Plot config kwargs to pass to ax.bar().

    Returns: None
    '''
    # Cast list of orientations to numpy array.
    orientations = np.array(orientations)
    # Force orientations to take the range -90 to 90 degrees to permit plotting of a half rose diagram.
    orientations[orientations > (np.pi/2)] -= np.pi
    orientations[orientations < -(np.pi/2)] += np.pi
    # Configure the half rose axis.
    ax.set_theta_direction(+1)
    ax.set_thetamin(-90)
    ax.set_thetamax(90)
    # Bin the orientations in the half rose range.
    # The range is expanded a bit to account for slight imprecision in radians conversion.
    counts,bins = np.histogram(orientations,bins=np.radians(np.linspace(-90.1,90.1,n_bins)))
    # Compute midpoints of each bin's angular range.
    midpoints = (bins[1:] + bins[:-1])/2
    # Compute width of each bin.
    w = midpoints[1] - midpoints[0]
    # Produce barplot with exactly non-overlapping bars.
    ax.bar(midpoints,counts,width=w,**plot_kwargs)
    return

# Declare list of desired colors to use for plotting rose plots in order of the maximum area filters.
colors = ["blue","green","grey"]
# Initialize a figure with stereonet axis.
fig = plt.figure(constrained_layout=True,figsize=(2,3))
ax = fig.add_subplot(111,projection="polar")
# Initialize list to hold all
all_orientations = []
# Iterate through increasing maximum grain area.
for i,max_area in enumerate(max_area_order):
    # Get orientations of grains that are smaller than the active max area.
    all_orientations.extend(rotations[max_area])
    # Get the desired color, or use a colormap where the list of desired colors is too short.
    try:
        c = colors[i]
    except IndexError:
        print("Warning: list of colors (n=%u) is shorter than the list of maximum area filters (n=%u). The remaining colors will be taken from the matplotlib tab10 colormap." % (len(colors),len(max_area_order)))
        c = mpl.colormaps["tab10"](i)

    # Plot half rose diagram of the active groups' feature orientations.
    plot_half_rose(all_orientations,ax,n_bins=15,color=c,zorder=(len(max_area_order)-i+10))

plt.show()

None

Modal orientations can be found from this distribution, and the dependence of the distribution on grain size also checked.

